In [184]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [186]:
f = open('pride_and_prejudice.txt', 'r')
data = f.read()
f.close()

In [187]:
def encrypt(text, key):
#convert letter to ascii indexes and back to text
    encrypted_text = []
    for char in text:
        if char.isalpha():
            if char.isupper():
                ind = ord(char) - ord('A')
                ind = (ind + key) % 26
                new_ind = ind + ord('A')
                new_char = chr(new_ind)
            else:
                ind = ord(char) - ord('a')
                ind = (ind + key) % 26
                new_ind = ind + ord('a')
                new_char = chr(new_ind)

            encrypted_text.append(new_char)
        else:
            encrypted_text.append(char)
    encrypted = ''.join(encrypted_text)

    return encrypted

In [188]:
print(encrypt('z', 2))

b


In [189]:
# encrypt yor text
encrypted_text = encrypt(data, key = 3)

In [190]:
all_char_data = set(data)
all_char_encrypt = set(encrypted_text)
all_char_data.update(all_char_encrypt)
vocab = sorted(list(all_char_data))


In [215]:
# convert data and encrypted_text to numbers according to vocab
data_converted = [vocab.index(x) for x in data]
#use first 512000 examples
data_converted = data_converted[:512000]
encrypt_converted = [vocab.index(x) for x in encrypted_text]
encrypt_converted = encrypt_converted [:512000]
y = torch.FloatTensor(data_converted).view(-1, 100).long()
x = torch.FloatTensor(encrypt_converted).view(-1, 100).long()
# create pairs of tensors

len_examples = len(y)
dataset = []
for i in range(len_examples):
     dataset.append([x[i], y[i]])
print(len_examples, len(y))

5120 5120


In [216]:
#creat test-train split 80 % test
train_tensor = dataset[:int(len_examples*0.8)]
test_tensor = dataset[int(len_examples*0.8):]
len(test_tensor)

1024

In [218]:
num_classes = len(vocab)
embedding_size = 10
hidden_size = 10

In [219]:
class RNN(torch.nn.Module):
        
    def __init__(self, num_classes = num_classes, embedding_dim = embedding_size, hidden_dim = hidden_size):
        super(RNN, self).__init__()
        self.embedding = torch.nn.Embedding(num_classes, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim) 
        self.linear = torch.nn.Linear(hidden_dim, num_classes)
        # Initialize hidden:
        #self.hidden = torch.zeros(1,1,hidden_dim)

    def forward(self, X):
        model_in = self.embedding(X)
        model_in = model_in.unsqueeze(1)
        out, hidden = self.rnn(model_in)
        out = self.linear(out)
        out = out.view(-1, num_classes).squeeze()
        out = out.unsqueeze(2)
        return out

In [220]:
model_RNN = RNN()
softmax = torch.nn.functional.softmax
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_RNN.parameters(), lr=0.01)
#model_RNN.to(device)
model_RNN

RNN(
  (embedding): Embedding(78, 10)
  (rnn): RNN(10, 10)
  (linear): Linear(in_features=10, out_features=78, bias=True)
)

In [221]:
num_epochs = 20
accuracies, max_accuracy = [], 0
for epoch in range(num_epochs):
    model_RNN.train()
    print('Epoch: {}'.format(epoch))
    for x, y in train_tensor:
        optimizer.zero_grad()
        prediction = model_RNN(x)
        #print(prediction.shape)
        #print(f'prediction {prediction}')
        y_in = y.unsqueeze(1)
        #print(f'y_in {y_in}')
        # Calculate loss
        loss = criterion(prediction, y_in) 
        # Backpropagate
        loss.backward()
        # Update weights
        optimizer.step()

    print('Loss: {:6.4f}'.format(loss.item()))
      

Epoch: 0
Loss: 0.0018
Epoch: 1
Loss: 0.0003
Epoch: 2
Loss: 0.0001
Epoch: 3
Loss: 0.0000
Epoch: 4
Loss: 0.0000
Epoch: 5
Loss: 0.0000
Epoch: 6
Loss: 0.0000
Epoch: 7
Loss: 0.0000
Epoch: 8
Loss: 0.0000
Epoch: 9
Loss: 0.0000
Epoch: 10
Loss: 0.0000
Epoch: 11
Loss: 0.0000
Epoch: 12
Loss: 0.0000
Epoch: 13
Loss: 0.0000
Epoch: 14
Loss: 0.0000
Epoch: 15
Loss: 0.0000
Epoch: 16
Loss: 0.0000
Epoch: 17
Loss: 0.0000
Epoch: 18
Loss: 0.0000
Epoch: 19
Loss: 0.0000


In [225]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in test_tensor:
            model_RNN.eval()
            scores = model_RNN(x)
            # Compute a softmax over the outputs
            predictions = softmax(scores, dim=1)
            # Choose the number with the maximum probability
            _, batch_out = predictions.max(dim=1)
            # Remove fake dimension
            batch_out = batch_out.squeeze(1)
            # Calculate accuracy
            matches += torch.eq(batch_out, y).sum().item()
            #print(batch_out, y)
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 100.00%


In [226]:
test_text = test_tensor[:1]

In [228]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in test_text:
            model_RNN.eval()
            scores = model_RNN(x)
            # Compute a softmax over the outputs
            predictions = softmax(scores, dim=1)
            # Choose the number with the maximum probability
            _, batch_out = predictions.max(dim=1)
            # Remove fake dimension
            batch_out = batch_out.squeeze(1)
            #print(batch_out, y)
            output = batch_out.tolist()
            real_y = y.tolist()
            input_x = x.tolist()
            encrypted_message = [vocab[idx] for idx in  input_x]
            model_output = [vocab[idx] for idx in  output]
            real_output = [vocab[idx] for idx in  real_y]
            encrypted_input = ''.join(encrypted_message)
            model = ''.join(model_output)
            real = ''.join(real_output)
            print('\n',encrypted_input[:20], '\n', model[:20], '\n', real[:20] )

            



 uh yhub gliihuhqw iu 
 re very different fr 
 re very different fr
